In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
#os.environ["TF_CPP_VMODULE"] = "gpu_process_state=10,gpu_cudamallocasync_allocator=10"
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        # allowing gpu memory growth
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


2021-12-13 20:02:49.939364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 20:02:50.053927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 20:02:50.054226: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 20:02:50.056114: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer
from transformers import InputExample, InputFeatures

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [4]:
#model.summary()

AttributeError: 'XLNetForSequenceClassification' object has no attribute 'summary'

In [5]:
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('Twitter_Data(Relabeled).csv')
dataset.columns=['clean_text','category']
dataset['category']=pd.to_numeric(dataset['category'],errors='coerce')
dataset=dataset.dropna(subset=['category'])
dataset['category']=dataset['category'].astype(int)
train , test =train_test_split(dataset,train_size=0.7)
train.head(10)

,clean_text,category
93947,from modi mocking congress nyay madhuri dixit ...,2
43000,people like raghuram rajan and amartya sen mak...,1
137678,the hallmark this modi shah duo unadulterated ...,2
103549,who wanted impeach the judge who brought emerg...,1
126674,modified team ark again for the better future ...,2
110259,you only modithe rest dont count,1
59673,that exactly the point “ have the capability s...,2
142187,their commission shops flourished during sonia...,2
59536,than modi announced every citizen india credit...,1
114895,public prosecutor opposes nirav modi’ bail ple...,1


In [7]:
test.head()

,clean_text,category
142240,election does not mean army will stop protecti...,2
43541,modi found his relatives mars,1
140987,please dont show clippings modi when kanhayya ...,1
54559,just blessed have like modination should grate...,1
91932,main short coming present govt govt jobs jobs ...,2


In [8]:
InputExample(guid=None,
             text_a= "Hello, world",
             text_b=None,
             label=1.0)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1.0)

In [9]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
    train_InputExamples = train.apply(lambda x: InputExample(guid=None,  # Globally unique ID for bookkeeping, unused in this case
                                                             text_a=x[DATA_COLUMN],
                                                             text_b=None,
                                                             label=x[LABEL_COLUMN]), axis=1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None,  # Globally unique ID for bookkeeping, unused in this case
                                                                 text_a=x[DATA_COLUMN],
                                                                 text_b=None,
                                                                 label=x[LABEL_COLUMN]), axis=1)

    return train_InputExamples, validation_InputExamples

    train_InputExamples, validation_InputExamples = convert_data_to_examples(train,
                                                                             test,
                                                                             'clean_text',
                                                                             'category')


def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []  # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        try:
            input_dict = tokenizer.encode_plus(
                e.text_a,
                add_special_tokens=True,
                max_length=max_length,  # truncates if len(s) > max_length
                return_token_type_ids=True,
                return_attention_mask=True,
                # pads to the right by default # CHECK THIS for pad_to_max_length
                pad_to_max_length=True,
                truncation=True
            )

            input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
                                                         input_dict["token_type_ids"], input_dict['attention_mask'])

            features.append(
                InputFeatures(
                    input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
                )
            )
        except:
            print("error produced by:")
            print(e)

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32,
         "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'clean_text'
LABEL_COLUMN = 'category'

In [10]:
len(dataset)

162973

In [11]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=2)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=1)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=0)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=1)


In [12]:
from sklearn import metrics
checkpoint_path = "training_2/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
try:
    model.load_weights(checkpoint_path)
except:
    print("No checkpoint found")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=5, validation_data=validation_data,
          callbacks=[cp_callback])

No checkpoint found


AttributeError: 'XLNetForSequenceClassification' object has no attribute 'compile'